In [56]:
import pandas as pd
from datetime import datetime
import numpy as np

In [57]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')


Date  Added                             Removed  \
                 Date Ticker                    Security  Ticker   
0     August 30, 2021   TECH                  Bio-Techne    MXIM   
1       July 21, 2021   MRNA                     Moderna    ALXN   
2        June 4, 2021    NaN                         NaN     HFC   
3        June 3, 2021    OGN               Organon & Co.     NaN   
4        May 14, 2021    CRL  Charles River Laboratories    FLIR   
..                ...    ...                         ...     ...   
273  December 5, 2000    SBL         Symbol Technologies      OI   
274  December 5, 2000    AYE            Allegheny Energy     GRA   
275  December 5, 2000    ABK             Ambac Financial     CCK   
276     July 27, 2000   JDSU                JDS Uniphase     RAD   
277  December 7, 1999   YHOO                      Yahoo!     LDW   

                              \
                    Security   
0           Maxim Integrated   
1    Alexion Pharmaceuticals   
2              HollyFrontier   
3                        NaN   
4               FLIR Systems   
..                       ...   
273           Owens-Illinois   
274                 WR Grace   
275           Crown Holdings   
276                  RiteAid   
277                  Laidlaw   

                                                Reason  
                                                Reason  
0    S&P 500 constituent Analog Devices acquired Ma...  
1    AstraZeneca Plc acquired Alexion Pharmaceutica...  
2                     Market capitalization change.[8]  
3    S&P 500/100 constituent Merck & Co. spun off O...  
4    S&P 500 constituent Teledyne Technologies acqu...  
..                                                 ...  
273                                Market Cap changes.  
274                                Market Cap changes.  
275                                Market Cap changes.  
276                            Market Cap change.[218]  
277                 Market capitalization change.[219]  

[278 rows x 6 columns]

In [58]:
# Get current S&P table and set header column
sp500 = data[0].iloc[1:,[0,1,6,7]]
columns = ['added_ticker', 'name', 'date', 'cik']
sp500.columns = columns
sp500.loc[sp500['date'].isnull(), 'date'] = '1957-01-01'

# One date is in the wrong format. Correcting it.
sp500.loc[~sp500['date'].str.match('\d{4}-\d{2}-\d{2}'), 'date'] = '1985-01-01'
#sp500.loc[:,'date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
sp500 = pd.melt(sp500, id_vars=['date', 'name', 'cik'], value_vars=['added_ticker'])
sp500.head()

,date,name,cik,variable,value
0,1964-03-31,Abbott Laboratories,1800,added_ticker,ABT
1,2012-12-31,AbbVie,1551152,added_ticker,ABBV
2,2018-05-31,Abiomed,815094,added_ticker,ABMD
3,2011-07-06,Accenture,1467373,added_ticker,ACN
4,2015-08-31,Activision Blizzard,718877,added_ticker,ATVI


In [59]:
sp500_adjustments = data[1]
sp500_adjustments = sp500_adjustments[2:].copy()
columns = ['date', 'added_ticker', 'added_name', 'removed_ticker', 'removed_name', 'reason']
sp500_adjustments.columns = columns
updates = sp500_adjustments[~sp500_adjustments['date'].str.contains(',')].T.shift(1).T
sp500_adjustments['date'].loc[~sp500_adjustments['date'].str.contains(',')] = np.nan
sp500_adjustments[sp500_adjustments['added_ticker'].isnull()]
sp500_adjustments.update(updates)
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_additions = sp500_adjustments[~sp500_adjustments['added_ticker'].isnull()]
sp500_additions = sp500_additions[['date', 'added_ticker', 'added_name']]
sp500_additions.rename(columns={'added_name': 'name'}, inplace=True)
sp500_additions = pd.melt(sp500_additions, id_vars=['date','name'], value_vars=['added_ticker'])
sp500_deletions = sp500_adjustments[~sp500_adjustments['removed_ticker'].isnull()]
sp500_deletions = sp500_deletions[['date', 'removed_ticker', 'removed_name']]
sp500_deletions.rename(columns={'removed_name': 'name'}, inplace=True)
sp500_deletions = pd.melt(sp500_deletions, id_vars=['date','name'], value_vars=['removed_ticker'])

sp500_history = pd.concat([sp500_deletions, sp500_additions])
sp500_history.head()

,date,name,variable,value
0,"June 4, 2021",HollyFrontier,removed_ticker,HFC
1,"May 14, 2021",FLIR Systems,removed_ticker,FLIR
2,"April 20, 2021",Varian Medical Systems,removed_ticker,VAR
3,"March 22, 2021",Flowserve,removed_ticker,FLS
4,"March 22, 2021",SL Green Realty,removed_ticker,SLG


,date,name,variable,value
0,"June 4, 2021",HollyFrontier,removed_ticker,HFC
1,"May 14, 2021",FLIR Systems,removed_ticker,FLIR
2,"April 20, 2021",Varian Medical Systems,removed_ticker,VAR
3,"March 22, 2021",Flowserve,removed_ticker,FLS
4,"March 22, 2021",SL Green Realty,removed_ticker,SLG
...,...,...,...,...
266,"December 5, 2000",Symbol Technologies,added_ticker,SBL
267,"December 5, 2000",Allegheny Energy,added_ticker,AYE
268,"December 5, 2000",Ambac Financial,added_ticker,ABK
269,"July 27, 2000",JDS Uniphase,added_ticker,JDSU


In [60]:
df = pd.concat([sp500, sp500_history], ignore_index=True)
#df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='cik', ascending=False, inplace=True)
deduped_df = df[~df.duplicated(['date', 'variable', 'value'])].copy()
deduped_df.sort_values(by='date',inplace=True)
deduped_df.to_csv("sp500_history.csv")
deduped_df.head()

,date,name,cik,variable,value
423,1957-01-01,State Street Corporation,93751.0,added_ticker,STT
390,1957-01-01,Qualcomm,804328.0,added_ticker,QCOM
135,1957-01-01,Danaher Corporation,313616.0,added_ticker,DHR
306,1957-01-01,McKesson Corporation,927653.0,added_ticker,MCK
203,1957-01-01,Franklin Resources,38777.0,added_ticker,BEN


,name,cik,ticker
499,Yum! Brands,1041061.0,YUM
501,Zimmer Biomet,1136869.0,ZBH
500,Zebra Technologies,877212.0,ZBRA
502,Zions Bancorp,109380.0,ZION
503,Zoetis,1555280.0,ZTS


In [61]:
deduped_df.sort_values(by='cik', ascending=False, inplace=True)
deduped_df = deduped_df[~deduped_df.duplicated('value')]
# discovery has 2 share classes listed
deduped_df = deduped_df[~deduped_df.duplicated('cik')]
deduped_df.sort_values(by='value', inplace=True)
deduped_df.drop(['date', 'variable'], axis=1, inplace=True)
deduped_df.rename(columns={'value':'ticker'}, inplace=True)
deduped_df.to_csv("sp500_constituents.csv")
deduped_df.head()

,name,cik,ticker
10,Agilent Technologies,1090872.0,A
26,American Airlines Group,6201.0,AAL
7,Advance Auto Parts,1158449.0,AAP
43,Apple,320193.0,AAPL
1,AbbVie,1551152.0,ABBV
